# Image-based visual servoing

We will create a classical image-based visual servoing system, which will drive the camera (mounted on the end of a Franka-Emika Panda robot) toward a goal comprising four 3D points.

Some references for this material are:

* [Visual servo control, Part I: Basic approaches, François Chaumette, S. Hutchinson](https://hal.inria.fr/inria-00350283/document)
* Robotics, Vision & Control, §15, Peter Corke, 2017

## Configuring CoLab

To make this run nicely in CoLab we have to install a bunch of packages.  This will be slow and at the end you will need to restart the kernel.  Push the `RESTART KERNEL` button that appears or from the menu above `Runtime/RestartRuntime`

In [ ]:
!pip install numpy --upgrade
!pip install roboticstoolbox-python
!pip install -U ipython

Now, we can import some of the packages we need throughout, NumPy, and configure the CoLab environment a bit.

In [ ]:
import numpy as np

# display result of assignments
%config ZMQInteractiveShell.ast_node_interactivity = 'last_expr_or_assign'
# make NumPy display a bit nicer
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:10.4g}" if abs(x) > 1e-10 else f"{0:10.4g}"})
# make cells nice and wide
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

Now import the classes we need from the Robotics, Machine Vision, and SpatialMaths Toolboxes

In [ ]:
from roboticstoolbox import models
from machinevisiontoolbox import CentralCamera, mkgrid
from spatialmath import SE3
from spatialmath.base import plotvol3, plot_sphere

We will use a 7-axis Franka-Emika Panda robot

In [ ]:
robot = models.DH.Panda()

and a perspective camera with default parameters, which will be listed

In [ ]:
camera = CentralCamera.Default()

which we see is a megapixel (1000x1000) image with an 8mm focal length lens.  Its field of view is

In [ ]:
camera.fov() * np.rad2deg(1)

of the order of 60°.

We will place four objects in the world, in a vertical plane

In [ ]:
P = mkgrid(side=0.4, n=2, pose=SE3(2, 0, 0.6) * SE3.Ry(90, unit='deg'))

`mkgrid` creates a 2x2 grid of 3D coordinates within a 0.4x0.4 area in the xy-plane.  The `pose` argument rotates that grid about the y-axis and translates it 2m out along the x-axis.  The result is four 3D points, one per column.

To visualize these is easy

In [ ]:
plotvol3([-1, 4, -2.5, 2.5, -1, 4])
plot_sphere(0.05, P)

The projection of these points for camera at the origin, looking along the x-axis is

In [ ]:
p = camera.project_point(P, pose=SE3.Ry(90, unit='deg'))

and these are the coordinates of the 3D points on the image plane with units of pixels.  Again, points correspond to columns in the array.

We can visualize this on the `camera` object's virtual image plane

In [ ]:
camera.plot_point(p);

Now let's say that the desired image-plane coordinates are

In [ ]:
pstar = mkgrid(n=2, side=700)[:2, :] + 500

The error between the current image plane coordinates and the desired is

In [ ]:
e = pstar - p

which we can reshape as a 1D array in column order

In [ ]:
e = e.flatten(order="F")

The image Jacobian, or interaction matrix is

In [ ]:
Jv = camera.visjac_p(p, depth=2)

We can't go into all the details of image-based visual servong here, but the Jacobian has a dependency on the depth of the points, their distance from the camera parallel to the optical axis.  In this case we have given a value of 2, but in practice this needs to be determined in an application specific way, or estimated online.

The camera velocity required to move toward the desired view is

In [ ]:
v = 0.01 * np.linalg.pinv(Jv) @ e

where we have added a scalar gain term.  We see is mostly velocity in the camera's x- and z-directions, toward the target.

The required robot joint velocity can be determined from the manipulator Jacobian, but first we need to know the joint configuration to place the camera at this pose

In [ ]:
iksol = robot.ikine_LMS(camera.pose)

which was successful. Now we can compute the manipulator Jacobian, in the end-effector frame

In [ ]:
J = robot.jacobe(iksol.q)

and then using resolved-rate motion control, to transform the desired camera velocity (in the camera frame) to joint velocity

In [ ]:
qd = np.linalg.pinv(J) @ v

This is the "guts" of an image-basd visual servoing system.  To make it complete we need to put the above lines into an integration loop.

If you manage to get that done (congratulations!) then other things to investigate would be:

* animate the robot using Swift
  * you could even add the small spheres into the Swift environment, using the `spatialgeometry` package, talk to us about this
* animate the display of points on the image plane
* experiment with different values of gain
* experiment with different initial camera poses (joint configurations) or target configurations
* experiment with different values of depth when computing the visual Jacobian.  If you're feeling ambitious, perhaps create a depth estimator.